In [1]:
#imports

import folium,math,matplotlib
import matplotlib.pyplot as plt
import pandas as pd,json
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin
from datetime import date, datetime, timedelta
from sklearn.pipeline import Pipeline
import branca.colormap as cm

In [2]:
###Functions from Dr. S's code

def parseTStamp(tstamp_df):    
    return datetime.strptime(tstamp_df['timeAtPosition'], '%Y-%m-%dT%H:%M:%S%z')


def dms2dec(dms_tuple):
   # print(dms_tuple)
    
    deg, minutes, seconds, direction = dms_tuple
    if np.isnan(seconds):
        seconds=0
    # print("Out", deg, minutes, seconds, direction)
    sign = 1
    direction = direction.strip()
    if direction == 'W' or direction == 'WEST' or direction =='S' or direction=='SOUTH':
        sign=-1
    return sign*(deg + minutes/60.0 + seconds/(60.0*60.0))


def simple2alt(alt_df):
    try:
        return int(alt_df['simpleAltitude'].strip('C').strip('T')) *100
    except Exception as exp:
        return alt_df['simpleAltitude']
    
    
class TFMSDataPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    
    def fit(self, X, y=None):
        return self
    
    
    def transform(self, X):        
        X['lat'] = X[['latitudeDMS_degrees','latitudeDMS_minutes','latitudeDMS_seconds', 'latitudeDMS_direction']].apply(dms2dec, axis=1)
        X['long'] = X[['longitudeDMS_degrees','longitudeDMS_minutes','longitudeDMS_seconds', 'longitudeDMS_direction']].apply(dms2dec, axis=1)
        X['alt'] = X[['simpleAltitude']].apply(simple2alt, axis=1)
        X['tstamp'] = X[['timeAtPosition']].apply(parseTStamp, axis=1)
        return X[self.attributes]
    

extract_attributes = [
    'tstamp',
    'flightRef',
    'acid',
    'gufi',    
    'lat',
    'long',
    'alt',
    'speed',
    'depArpt',
    'arrArpt']

In [3]:
tfms_csv_files = ['KDFW20190512_170000.csv','KDFW20190512_160000.csv']

In [4]:
tfms_df_dict={}

mymap = folium.Map(location=[32.8917,-97.0417], zoom_start=9 , tiles='Stamen Terrain')

In [5]:
def clean_df(data):
    waypoints_df=None
    pipe = Pipeline([('SELECT', TFMSDataPreprocessor(extract_attributes)), ])
    waypoints_df = pipe.fit_transform(data)
    waypoints_df=waypoints_df.dropna()
    return waypoints_df

In [6]:
tfms_df_dict={}
colors=['b','c','g','y','r']
min_alt=[]
max_alt=[]
lats=[]
longs=[]
altitudes=[]
min_alt=[]
max_alt=[]

flt_waypoints_df=None

for csv in tfms_csv_files:
    tfms_df_dict[csv]=pd.read_csv(csv)
    #fg=folium.FeatureGroup(name=csv, show=False)
    
for key in tfms_df_dict:
    flt_waypoints_df=clean_df(tfms_df_dict[key])
    
    lats.append(flt_waypoints_df['lat'].tolist())
    longs.append(flt_waypoints_df['long'].tolist())
    altitudes.append(flt_waypoints_df['alt'].tolist())
    
for ii in range(0,len(altitudes)):
    min_alt.append(min(altitudes[ii]))
    max_alt.append(max(altitudes[ii]))
    
vmin=min(min_alt)
vmax=max(max_alt)
cmap=cm.LinearColormap(colors=colors, vmin=vmin, vmax=vmax)   
cmap.caption='Flight altitude'
 
for key in tfms_df_dict:
    #cmap=cm.LinearColormap(colors=colors, vmin=vmin, vmax=vmax)
    fg=folium.FeatureGroup(name=key, show=False)
    
    if key==tfms_csv_files[0]:
        for ii in range(0,len(altitudes[0])):
            #color_interp = np.interp(altitudes[ii],[alt_min,alt_max],[50,200])
            fg.add_child(folium.CircleMarker([lats[0][ii],longs[0][ii]],
                #radius=0.0005*altitudes[ii],
                radius=5,
                popup=altitudes[0][ii],
                          fill=True,
                fill_color=cmap(altitudes[0][ii]),
                weight=0, fill_opacity=1))
        
           
    if key==tfms_csv_files[1]:
        for ii in range(0,len(altitudes[1])):
            pt=(lats[1][ii],longs[1][ii])
            W=pt[1]-0.0005
            E=pt[1]+0.0005
            N=pt[0]+0.0005
            S=pt[0]-0.0005
            
            upper_left=(N,W)
            upper_right=(N,E)
            lower_right=(S,E)
            lower_left=(S,W)
            
            edges = [upper_left, upper_right, lower_right, lower_left]
            #bounds=[edges[2:],edges[:2]]
            #print(bounds)
            #color_interp = np.interp(altitudes[ii],[alt_min,alt_max],[50,200])
            fg.add_child(folium.Rectangle(
                bounds=[edges[2:],edges[:2]],
                popup=altitudes[1][ii],
                fill=True,
                color=cmap(altitudes[1][ii]),
                fill_color=cmap(altitudes[1][ii]),
                weight=3,fill_opacity=1))

    mymap.add_child(fg)

mymap.add_child(cmap)
mymap.add_child(folium.map.LayerControl())
mymap.save('MyFoliummap_Icons.html')